In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
import os
import math
from time import time
import datetime
from keras.callbacks import TensorBoard, TerminateOnNaN, LearningRateScheduler, Callback
import tensorflow as tf
from keras import backend as K
import multiprocessing as mp
import numpy as np


Using TensorFlow backend.


# Preprocessing

In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Tensorboard

In [2]:
tensorboard = TensorBoard(log_dir="logs/0/{}".format(time()))

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [7]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 14s 9ms/step - loss: 1.8878 - acc: 0.3090 - val_loss: 1.5725 - val_acc: 0.4348
Epoch 2/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.6016 - acc: 0.4136 - val_loss: 1.4437 - val_acc: 0.4786
Epoch 3/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.4777 - acc: 0.4632 - val_loss: 1.3218 - val_acc: 0.5266
Epoch 4/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.3941 - acc: 0.5006 - val_loss: 1.2258 - val_acc: 0.5640
Epoch 5/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.3291 - acc: 0.5239 - val_loss: 1.1574 - val_acc: 0.5872
Epoch 6/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.2739 - acc: 0.5469 - val_loss: 1.1615 - val_acc: 0.5887
Epoch 7/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.2248 - acc: 0.5676 - val_loss: 1.1431 - val_acc: 0.5934
Epoch 8/100
1

In [8]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('Runtime:', str(end-start))

10000/10000 [==============================] - 2s 155us/step
Test loss: 0.7649821676254273
Test accuracy: 0.7514
Runtime: 1324.4300031661987


# Terminate on NaN

In [13]:
tensorboard = TensorBoard(log_dir="logs/nan/{}".format(time()))
terminateonnan = TerminateOnNaN()

In [14]:
model_nan = Sequential()
model_nan.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_nan.add(Activation('relu'))
model_nan.add(Conv2D(32, (3, 3)))
model_nan.add(Activation('relu'))
model_nan.add(MaxPooling2D(pool_size=(2, 2)))
model_nan.add(Dropout(0.25))

model_nan.add(Conv2D(64, (3, 3), padding='same'))
model_nan.add(Activation('relu'))
model_nan.add(Conv2D(64, (3, 3)))
model_nan.add(Activation('relu'))
model_nan.add(MaxPooling2D(pool_size=(2, 2)))
model_nan.add(Dropout(0.25))

model_nan.add(Flatten())
model_nan.add(Dense(512))
model_nan.add(Activation('relu'))
model_nan.add(Dropout(0.5))
model_nan.add(Dense(num_classes))
model_nan.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_nan.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [15]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model_nan.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard, terminateonnan])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_nan.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard, terminateonnan])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 13s 9ms/step - loss: 1.8813 - acc: 0.3093 - val_loss: 1.6201 - val_acc: 0.4193
Epoch 2/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.5889 - acc: 0.4195 - val_loss: 1.6024 - val_acc: 0.4368
Epoch 3/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.4626 - acc: 0.4698 - val_loss: 1.5070 - val_acc: 0.4663
Epoch 4/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.3717 - acc: 0.5085 - val_loss: 1.2584 - val_acc: 0.5580
Epoch 5/100
1563/1562 [==============================] - 13s 9ms/step - loss: 1.2983 - acc: 0.5397 - val_loss: 1.2081 - val_acc: 0.5698
Epoch 6/100
1563/1562 [==============================] - 13s 9ms/step - loss: 1.2357 - acc: 0.5599 - val_loss: 1.0915 - val_acc: 0.6127
Epoch 7/100
1563/1562 [==============================] - 14s 9ms/step - loss: 1.1866 - acc: 0.5795 - val_loss: 1.0496 - val_acc: 0.6273
Epoch 8/100
1

In [16]:
# Score trained model.
scores = model_nan.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('Runtime:', str(end-start))

10000/10000 [==============================] - 2s 158us/step
Test loss: 0.7133792991638184
Test accuracy: 0.7628
Runtime: 1326.8641436100006


### Guaranteeing a NaN Loss
To test when this callback gets triggered, we need to guarantee that a NaN loss will occur. We can do this by replaceing some of the values in the training data with NaNs.

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network

In [33]:
x_train_nan = x_train.copy()
x_train_nan[x_train_nan >= 0] = np.nan
x_train_nan.shape

(50000, 32, 32, 3)

In [34]:
np.isnan(x_train_nan).any()

True

In [36]:
tensorboard = TensorBoard(log_dir="logs/nan/{}".format(time()))
terminateonnan = TerminateOnNaN()

model_nan_2 = Sequential()
model_nan_2.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_nan_2.add(Activation('relu'))
model_nan_2.add(Conv2D(32, (3, 3)))
model_nan_2.add(Activation('relu'))
model_nan_2.add(MaxPooling2D(pool_size=(2, 2)))
model_nan_2.add(Dropout(0.25))

model_nan_2.add(Conv2D(64, (3, 3), padding='same'))
model_nan_2.add(Activation('relu'))
model_nan_2.add(Conv2D(64, (3, 3)))
model_nan_2.add(Activation('relu'))
model_nan_2.add(MaxPooling2D(pool_size=(2, 2)))
model_nan_2.add(Dropout(0.25))

model_nan_2.add(Flatten())
model_nan_2.add(Dense(512))
model_nan_2.add(Activation('relu'))
model_nan_2.add(Dropout(0.5))
model_nan_2.add(Dense(num_classes))
model_nan_2.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6) 

# Let's train the model using RMSprop
model_nan_2.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [37]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model_nan_2.fit(x_train_nan, y_train,
              batch_size=batch_size,
              epochs=300,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard, terminateonnan])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_nan_2.fit_generator(datagen.flow(x_train_nan, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard, terminateonnan])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 13s 8ms/step - loss: 2.3026 - acc: 0.0976 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 2/100
1563/1562 [==============================] - 13s 9ms/step - loss: 2.3026 - acc: 0.0969 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 3/100
1563/1562 [==============================] - 13s 9ms/step - loss: 2.3026 - acc: 0.0971 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 4/100
1563/1562 [==============================] - 14s 9ms/step - loss: 2.3026 - acc: 0.0964 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 5/100
1563/1562 [==============================] - 14s 9ms/step - loss: 2.3026 - acc: 0.0936 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 6/100
1563/1562 [==============================] - 13s 8ms/step - loss: 2.3026 - acc: 0.0969 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 7/100
1563/1562 [==============================] - 13s 9ms/step - loss: 2.3026 - acc: 0.0970 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 8/100
1

1563/1562 [==============================] - 14s 9ms/step - loss: 2.3026 - acc: 0.0984 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 61/100
1563/1562 [==============================] - 13s 9ms/step - loss: 2.3026 - acc: 0.0970 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 62/100
1563/1562 [==============================] - 13s 8ms/step - loss: 2.3026 - acc: 0.0980 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 63/100
1563/1562 [==============================] - 13s 9ms/step - loss: 2.3026 - acc: 0.0978 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 64/100
1563/1562 [==============================] - 13s 9ms/step - loss: 2.3026 - acc: 0.0972 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 65/100
1563/1562 [==============================] - 13s 9ms/step - loss: 2.3026 - acc: 0.0988 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 66/100
1563/1562 [==============================] - 13s 9ms/step - loss: 2.3026 - acc: 0.0968 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 67/100
1563/1562 [==============================

In [38]:
# Score trained model.
scores = model_nan_2.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('Runtime:', str(end-start))

10000/10000 [==============================] - 2s 163us/step
Test loss: 2.302585478210449
Test accuracy: 0.1
Runtime: 1342.4824059009552


# Learning Rate Scheduler

In [18]:
tensorboard = TensorBoard(log_dir="logs/lr_scheduler/{}".format(time()))

# decay schedule adapted from https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
def schedule(epoch, lr):
    drop = 0.9
    epochs_drop = 10.0
    return lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    
lrscheduler = LearningRateScheduler(schedule, verbose=1)

In [19]:
model_lr = Sequential()
model_lr.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_lr.add(Activation('relu'))
model_lr.add(Conv2D(32, (3, 3)))
model_lr.add(Activation('relu'))
model_lr.add(MaxPooling2D(pool_size=(2, 2)))
model_lr.add(Dropout(0.25))

model_lr.add(Conv2D(64, (3, 3), padding='same'))
model_lr.add(Activation('relu'))
model_lr.add(Conv2D(64, (3, 3)))
model_lr.add(Activation('relu'))
model_lr.add(MaxPooling2D(pool_size=(2, 2)))
model_lr.add(Dropout(0.25))

model_lr.add(Flatten())
model_lr.add(Dense(512))
model_lr.add(Activation('relu'))
model_lr.add(Dropout(0.5))
model_lr.add(Dense(num_classes))
model_lr.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001) # , decay=1e-6

# Let's train the model using RMSprop
model_lr.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [20]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model_lr.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard, lrscheduler])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_lr.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard, lrscheduler])

Using real-time data augmentation.
Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
1563/1562 [==============================] - 14s 9ms/step - loss: 1.8878 - acc: 0.3081 - val_loss: 1.6535 - val_acc: 0.4140
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
1563/1562 [==============================] - 13s 9ms/step - loss: 1.6142 - acc: 0.4085 - val_loss: 1.5365 - val_acc: 0.4447
Epoch 3/100

Epoch 00003: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
1563/1562 [==============================] - 13s 8ms/step - loss: 1.4799 - acc: 0.4604 - val_loss: 1.3396 - val_acc: 0.5101
Epoch 4/100

Epoch 00004: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
1563/1562 [==============================] - 13s 9ms/step - loss: 1.3988 - acc: 0.4959 - val_loss: 1.2556 - val_acc: 0.5510
Epoch 5/100

Epoch 00005: LearningRateScheduler setting learning rate to 9.9999997473

Epoch 38/100

Epoch 00038: LearningRateScheduler setting learning rate to 2.465034885972273e-07.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.9127 - acc: 0.6777 - val_loss: 0.8518 - val_acc: 0.7033
Epoch 39/100

Epoch 00039: LearningRateScheduler setting learning rate to 1.7970103357356496e-07.
1563/1562 [==============================] - 13s 9ms/step - loss: 0.9175 - acc: 0.6794 - val_loss: 0.8512 - val_acc: 0.7036
Epoch 40/100

Epoch 00040: LearningRateScheduler setting learning rate to 1.1790184756819144e-07.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.9116 - acc: 0.6811 - val_loss: 0.8514 - val_acc: 0.7039
Epoch 41/100

Epoch 00041: LearningRateScheduler setting learning rate to 7.735540265940699e-08.
1563/1562 [==============================] - 13s 9ms/step - loss: 0.9158 - acc: 0.6806 - val_loss: 0.8515 - val_acc: 0.7035
Epoch 42/100

Epoch 00042: LearningRateScheduler setting learning rate to 5.0752879307225384e-08.
1563/1562 [=====

1563/1562 [==============================] - 13s 9ms/step - loss: 0.9147 - acc: 0.6813 - val_loss: 0.8507 - val_acc: 0.7040
Epoch 76/100

Epoch 00076: LearningRateScheduler setting learning rate to 1.4086720522650144e-16.
1563/1562 [==============================] - 13s 8ms/step - loss: 0.9151 - acc: 0.6784 - val_loss: 0.8507 - val_acc: 0.7040
Epoch 77/100

Epoch 00077: LearningRateScheduler setting learning rate to 6.737634696408825e-17.
1563/1562 [==============================] - 13s 9ms/step - loss: 0.9155 - acc: 0.6793 - val_loss: 0.8507 - val_acc: 0.7040
Epoch 78/100

Epoch 00078: LearningRateScheduler setting learning rate to 3.222589888475881e-17.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.9203 - acc: 0.6769 - val_loss: 0.8507 - val_acc: 0.7040
Epoch 79/100

Epoch 00079: LearningRateScheduler setting learning rate to 1.5413547052794303e-17.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.9127 - acc: 0.6813 - val_loss: 0.8507 - val_ac

In [21]:
# Score trained model.
scores = model_lr.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('Runtime:', str(end-start))

10000/10000 [==============================] - 2s 160us/step
Test loss: 0.8506660752296448
Test accuracy: 0.704
Runtime: 1343.6695251464844


# Custom Callbacks

In [26]:
tensorboard = TensorBoard(log_dir="logs/custom/{}".format(time()))

# Example from https://keras.io/callbacks/
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        
losshistory = LossHistory()

In [27]:
model_custom = Sequential()
model_custom.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_custom.add(Activation('relu'))
model_custom.add(Conv2D(32, (3, 3)))
model_custom.add(Activation('relu'))
model_custom.add(MaxPooling2D(pool_size=(2, 2)))
model_custom.add(Dropout(0.25))

model_custom.add(Conv2D(64, (3, 3), padding='same'))
model_custom.add(Activation('relu'))
model_custom.add(Conv2D(64, (3, 3)))
model_custom.add(Activation('relu'))
model_custom.add(MaxPooling2D(pool_size=(2, 2)))
model_custom.add(Dropout(0.25))

model_custom.add(Flatten())
model_custom.add(Dense(512))
model_custom.add(Activation('relu'))
model_custom.add(Dropout(0.5))
model_custom.add(Dense(num_classes))
model_custom.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6) 

# Let's train the model using RMSprop
model_custom.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [28]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model_custom.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard, losshistory])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_custom.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard, losshistory])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 14s 9ms/step - loss: 1.8719 - acc: 0.3087 - val_loss: 1.5517 - val_acc: 0.4406
Epoch 2/100
1563/1562 [==============================] - 14s 9ms/step - loss: 1.5771 - acc: 0.4193 - val_loss: 1.4522 - val_acc: 0.4808
Epoch 3/100
1563/1562 [==============================] - 14s 9ms/step - loss: 1.4605 - acc: 0.4676 - val_loss: 1.3972 - val_acc: 0.4927
Epoch 4/100
1563/1562 [==============================] - 13s 8ms/step - loss: 1.3740 - acc: 0.5078 - val_loss: 1.2830 - val_acc: 0.5477
Epoch 5/100
1563/1562 [==============================] - 13s 9ms/step - loss: 1.3051 - acc: 0.5330 - val_loss: 1.2099 - val_acc: 0.5710
Epoch 6/100
1563/1562 [==============================] - 14s 9ms/step - loss: 1.2471 - acc: 0.5566 - val_loss: 1.1356 - val_acc: 0.5972
Epoch 7/100
1563/1562 [==============================] - 13s 9ms/step - loss: 1.1981 - acc: 0.5743 - val_loss: 1.1505 - val_acc: 0.5935
Epoch 8/100
1

In [29]:
print(losshistory.losses)

[2.3451781, 2.2898335, 2.3097787, 2.285324, 2.3232164, 2.296553, 2.2889848, 2.2912643, 2.2978754, 2.3165379, 2.3292284, 2.3229232, 2.332654, 2.2930734, 2.315714, 2.3411372, 2.2935064, 2.2925484, 2.2936068, 2.3126936, 2.2976334, 2.2727056, 2.3210983, 2.309294, 2.2976635, 2.2749472, 2.2717102, 2.285037, 2.2819738, 2.3044312, 2.295658, 2.2842374, 2.3418126, 2.3070555, 2.2933989, 2.3090186, 2.2910972, 2.2963629, 2.3068137, 2.296558, 2.3261833, 2.2938652, 2.2919254, 2.3071947, 2.2555735, 2.326353, 2.2834296, 2.3140798, 2.2961266, 2.2666984, 2.2795386, 2.2751698, 2.2945352, 2.2660089, 2.2197013, 2.289999, 2.3490124, 2.3531585, 2.28486, 2.2679143, 2.2990327, 2.2598495, 2.2455344, 2.3502173, 2.2851923, 2.2619162, 2.3041801, 2.2716928, 2.2897806, 2.3057585, 2.305815, 2.262783, 2.3182595, 2.3326488, 2.2454364, 2.2600517, 2.3255298, 2.2831776, 2.2754986, 2.2673552, 2.2762022, 2.2793233, 2.2858722, 2.2538147, 2.3345137, 2.2674353, 2.269051, 2.2861621, 2.2947426, 2.2490458, 2.2803817, 2.2736723, 2.

In [30]:
# Score trained model.
scores = model_custom.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('Runtime:', str(end-start))

10000/10000 [==============================] - 1s 140us/step
Test loss: 0.717925118303299
Test accuracy: 0.7592
Runtime: 1349.572916984558


As a second example, let's look at tracking the learning rate scheduler:

In [47]:
tensorboard = TensorBoard(log_dir="logs/custom/{}".format(time()))

# Method from https://github.com/keras-team/keras/issues/7874
class LearningRateHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.lr_history = []
        
    def on_epoch_end(self, epoch, logs=None):
        self.lr_history.append(K.eval(self.model.optimizer.lr))
        
lrhistory = LearningRateHistory()

def schedule(epoch, lr):
    drop = 0.9
    epochs_drop = 10.0
    return lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    
lrscheduler = LearningRateScheduler(schedule, verbose=1)

In [48]:
model_custom = Sequential()
model_custom.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_custom.add(Activation('relu'))
model_custom.add(Conv2D(32, (3, 3)))
model_custom.add(Activation('relu'))
model_custom.add(MaxPooling2D(pool_size=(2, 2)))
model_custom.add(Dropout(0.25))

model_custom.add(Conv2D(64, (3, 3), padding='same'))
model_custom.add(Activation('relu'))
model_custom.add(Conv2D(64, (3, 3)))
model_custom.add(Activation('relu'))
model_custom.add(MaxPooling2D(pool_size=(2, 2)))
model_custom.add(Dropout(0.25))

model_custom.add(Flatten())
model_custom.add(Dense(512))
model_custom.add(Activation('relu'))
model_custom.add(Dropout(0.5))
model_custom.add(Dense(num_classes))
model_custom.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6) 

# Let's train the model using RMSprop
model_custom.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [49]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model_custom.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard, lrhistory, lrscheduler])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_custom.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard, lrhistory, lrscheduler])

Using real-time data augmentation.
Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
1563/1562 [==============================] - 14s 9ms/step - loss: 1.8926 - acc: 0.3025 - val_loss: 1.6124 - val_acc: 0.4091
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
1563/1562 [==============================] - 13s 9ms/step - loss: 1.5897 - acc: 0.4176 - val_loss: 1.4377 - val_acc: 0.4741
Epoch 3/100

Epoch 00003: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
1563/1562 [==============================] - 14s 9ms/step - loss: 1.4656 - acc: 0.4668 - val_loss: 1.3211 - val_acc: 0.5243
Epoch 4/100

Epoch 00004: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
1563/1562 [==============================] - 14s 9ms/step - loss: 1.3714 - acc: 0.5089 - val_loss: 1.2447 - val_acc: 0.5548
Epoch 5/100

Epoch 00005: LearningRateScheduler setting learning rate to 9.9999997473

Epoch 38/100

Epoch 00038: LearningRateScheduler setting learning rate to 2.465034885972273e-07.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.8974 - acc: 0.6892 - val_loss: 0.8167 - val_acc: 0.7141
Epoch 39/100

Epoch 00039: LearningRateScheduler setting learning rate to 1.7970103357356496e-07.
1563/1562 [==============================] - 13s 9ms/step - loss: 0.9071 - acc: 0.6837 - val_loss: 0.8159 - val_acc: 0.7144
Epoch 40/100

Epoch 00040: LearningRateScheduler setting learning rate to 1.1790184756819144e-07.
1563/1562 [==============================] - 13s 9ms/step - loss: 0.8993 - acc: 0.6869 - val_loss: 0.8166 - val_acc: 0.7137
Epoch 41/100

Epoch 00041: LearningRateScheduler setting learning rate to 7.735540265940699e-08.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.9002 - acc: 0.6848 - val_loss: 0.8168 - val_acc: 0.7139
Epoch 42/100

Epoch 00042: LearningRateScheduler setting learning rate to 5.0752879307225384e-08.
1563/1562 [=====

1563/1562 [==============================] - 14s 9ms/step - loss: 0.9007 - acc: 0.6843 - val_loss: 0.8165 - val_acc: 0.7139
Epoch 76/100

Epoch 00076: LearningRateScheduler setting learning rate to 1.4086720522650144e-16.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.9012 - acc: 0.6840 - val_loss: 0.8165 - val_acc: 0.7139
Epoch 77/100

Epoch 00077: LearningRateScheduler setting learning rate to 6.737634696408825e-17.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.9043 - acc: 0.6845 - val_loss: 0.8165 - val_acc: 0.7139
Epoch 78/100

Epoch 00078: LearningRateScheduler setting learning rate to 3.222589888475881e-17.
1563/1562 [==============================] - 13s 9ms/step - loss: 0.9007 - acc: 0.6865 - val_loss: 0.8165 - val_acc: 0.7139
Epoch 79/100

Epoch 00079: LearningRateScheduler setting learning rate to 1.5413547052794303e-17.
1563/1562 [==============================] - 14s 9ms/step - loss: 0.8954 - acc: 0.6865 - val_loss: 0.8165 - val_ac

In [50]:
# Score trained model.
scores = model_custom.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('Runtime:', str(end-start))

10000/10000 [==============================] - 2s 168us/step
Test loss: 0.8165272733688355
Test accuracy: 0.7139
Runtime: 1359.904479265213


In [51]:
print(lrhistory.lr_history)

[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 9e-05, 8.1e-05, 7.29e-05, 6.561e-05, 5.9049e-05, 5.31441e-05, 4.782969e-05, 4.304672e-05, 3.8742048e-05, 3.4867844e-05, 2.8242954e-05, 2.2876793e-05, 1.8530201e-05, 1.5009463e-05, 1.2157665e-05, 9.8477085e-06, 7.976644e-06, 6.4610817e-06, 5.2334763e-06, 4.2391157e-06, 3.0903154e-06, 2.25284e-06, 1.6423204e-06, 1.1972515e-06, 8.7279636e-07, 6.3626857e-07, 4.638398e-07, 3.3813922e-07, 2.4650348e-07, 1.7970103e-07, 1.1790185e-07, 7.73554e-08, 5.075288e-08, 3.3298964e-08, 2.184745e-08, 1.4334112e-08, 9.40461e-09, 6.170365e-09, 4.0483767e-09, 2.65614e-09, 1.5684241e-09, 9.261387e-10, 5.4687566e-10, 3.229246e-10, 1.9068375e-10, 1.1259685e-10, 6.6487316e-11, 3.9260095e-11, 2.3182694e-11, 1.3689149e-11, 7.274975e-12, 3.86622e-12, 2.0546676e-12, 1.0919346e-12, 5.8029883e-13, 3.0839459e-13, 1.6389353e-13, 8.709974e-14, 4.6288372e-14, 2.459954e-14, 1.1765883e-14, 5.6275853e-15, 2.6916566e-15, 1.287411e-15, 6.157647e-16, 2.9451833e-16